In [1]:
def set_paths():
    '''
    Deployment should be something like "dirs/dir1/use-cases"
    This function adds to the path "dirs/dir1/use-cases" and "dirs/dir1/"
    :return:
    '''
    import imp
    from os.path import dirname
    import os
    import sys

    USE_CASES = "/var/SP/data/home/asaezco/src/devel2/use-cases"#dirname(os.path.abspath(imp.find_module('churn')[1]))
    sys.path.append("/var/SP/data/home/asaezco/src/devel2/pykhaos")
    if USE_CASES not in sys.path:
        sys.path.append(USE_CASES)
        print("Added '{}' to path".format(USE_CASES))

    # if deployment is correct, this path should be the one that contains "use-cases", "pykhaos", ...
    # FIXME another way of doing it more general?
    DEVEL_SRC = os.path.dirname(USE_CASES)  # dir before use-cases dir
    if DEVEL_SRC not in sys.path:
        sys.path.append(DEVEL_SRC)
        print("Added '{}' to path".format(DEVEL_SRC))
set_paths()

Added '/var/SP/data/home/asaezco/src/devel2/use-cases' to path
Added '/var/SP/data/home/asaezco/src/devel2' to path


In [2]:
# coding=utf-8

import sys

from common.src.main.python.utils.hdfs_generic import *
import argparse
import os
import sys
import time
from pyspark.sql.functions import (udf,
                                    col,
                                    decode,
                                    when,
                                    lit,
                                    lower,
                                    concat,
                                    translate,
                                    count,
                                    sum as sql_sum,
                                    max as sql_max,
                                    min as sql_min,
                                    avg as sql_avg,
                                    greatest,
                                    least,
                                    isnull,
                                    isnan,
                                    struct, 
                                    substring,
                                    size,
                                    length,
                                    year,
                                    month,
                                    dayofmonth,
                                    unix_timestamp,
                                    date_format,
                                    from_unixtime,
                                    datediff,
                                    to_date, 
                                    desc,
                                    asc,
                                    countDistinct,
                                    row_number,
                                    skewness,
                                    kurtosis,
                                    concat_ws)

from pyspark.sql import Row, DataFrame, Column, Window
from pyspark.sql.types import DoubleType, StringType, IntegerType, DateType, ArrayType
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler, SQLTransformer, OneHotEncoder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from datetime import datetime
from itertools import chain
import numpy as np
from functools import reduce
from utils_general import *
from pykhaos.utils.date_functions import convert_to_date
from utils_model import *
from utils_fbb_churn import *
from metadata_fbb_churn import *
from feature_selection_utils import *
import subprocess


In [3]:
# Para que no haga falta recargar el notebook si hacemos un cambio en alguna de las funciones que estamos importando
%load_ext autoreload
%autoreload 2

In [4]:
# set BDP parameters
def setting_bdp(min_n_executors = 3, max_n_executors = 15, n_cores = 8, executor_memory = "12g", driver_memory="8g",
                   app_name = "Python app", driver_overhead="1g", executor_overhead='3g'):

    MAX_N_EXECUTORS = max_n_executors
    MIN_N_EXECUTORS = min_n_executors
    N_CORES_EXECUTOR = n_cores
    EXECUTOR_IDLE_MAX_TIME = 120
    EXECUTOR_MEMORY = executor_memory
    DRIVER_MEMORY = driver_memory
    N_CORES_DRIVER = 1
    MEMORY_OVERHEAD = N_CORES_EXECUTOR * 2048
    QUEUE = "root.BDPtenants.es.medium"
    BDA_CORE_VERSION = "1.0.0"

    SPARK_COMMON_OPTS = os.environ.get('SPARK_COMMON_OPTS', '')
    SPARK_COMMON_OPTS += " --executor-memory %s --driver-memory %s" % (EXECUTOR_MEMORY, DRIVER_MEMORY)
    SPARK_COMMON_OPTS += " --conf spark.shuffle.manager=tungsten-sort"
    SPARK_COMMON_OPTS += "  --queue %s" % QUEUE

    # Dynamic allocation configuration
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.enabled=true"
    SPARK_COMMON_OPTS += " --conf spark.shuffle.service.enabled=true"
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.maxExecutors=%s" % (MAX_N_EXECUTORS)
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.minExecutors=%s" % (MIN_N_EXECUTORS)
    SPARK_COMMON_OPTS += " --conf spark.executor.cores=%s" % (N_CORES_EXECUTOR)
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.executorIdleTimeout=%s" % (EXECUTOR_IDLE_MAX_TIME)
    # SPARK_COMMON_OPTS += " --conf spark.ui.port=58235"
    SPARK_COMMON_OPTS += " --conf spark.port.maxRetries=100"
    SPARK_COMMON_OPTS += " --conf spark.app.name='%s'" % (app_name)
    SPARK_COMMON_OPTS += " --conf spark.submit.deployMode=client"
    SPARK_COMMON_OPTS += " --conf spark.ui.showConsoleProgress=true"
    SPARK_COMMON_OPTS += " --conf spark.sql.broadcastTimeout=1200"
    SPARK_COMMON_OPTS += " --conf spark.yarn.executor.memoryOverhead={}".format(executor_overhead)
    SPARK_COMMON_OPTS += " --conf spark.yarn.executor.driverOverhead={}".format(driver_overhead)

    BDA_ENV = os.environ.get('BDA_USER_HOME', '')

    # Attach bda-core-ra codebase
    SPARK_COMMON_OPTS+=" --files {}/scripts/properties/red_agent/nodes.properties,{}/scripts/properties/red_agent/nodes-de.properties,{}/scripts/properties/red_agent/nodes-es.properties,{}/scripts/properties/red_agent/nodes-ie.properties,{}/scripts/properties/red_agent/nodes-it.properties,{}/scripts/properties/red_agent/nodes-pt.properties,{}/scripts/properties/red_agent/nodes-uk.properties".format(*[BDA_ENV]*7)

    os.environ["SPARK_COMMON_OPTS"] = SPARK_COMMON_OPTS
    os.environ["PYSPARK_SUBMIT_ARGS"] = "%s pyspark-shell " % SPARK_COMMON_OPTS

In [5]:
def get_spark_session(app_name="default name", log_level='INFO', min_n_executors = 3, max_n_executors = 15, n_cores = 4, executor_memory = "12g", driver_memory="8g"):
    HOME_SRC = os.path.join(os.environ.get('BDA_USER_HOME', ''), "src")
    if HOME_SRC not in sys.path:
        sys.path.append(HOME_SRC)


    setting_bdp(app_name=app_name, min_n_executors = min_n_executors, max_n_executors = max_n_executors, n_cores = n_cores, executor_memory = executor_memory, driver_memory=driver_memory)
    from common.src.main.python.utils.hdfs_generic import run_sc
    sc, spark, sql_context = run_sc(log_level=log_level)


    return sc, spark, sql_context

In [6]:
def initialize(app_name, min_n_executors = 3, max_n_executors = 15, n_cores = 4, executor_memory = "12g", driver_memory="8g"):
    import time
    start_time = time.time()

    print("_initialize spark")
    #import pykhaos.utils.pyspark_configuration as pyspark_config
    sc, spark, sql_context = get_spark_session(app_name=app_name, log_level="OFF", min_n_executors = min_n_executors, max_n_executors = max_n_executors, n_cores = n_cores,
                             executor_memory = executor_memory, driver_memory=driver_memory)
    print("Ended spark session: {} secs | default parallelism={}".format(time.time() - start_time,
                                                                         sc.defaultParallelism))
    return spark

In [7]:
spark = initialize("FBB Improvement ",executor_memory = "32g",min_n_executors = 6,max_n_executors = 15)


_initialize spark
Ended spark session: 35.352255106 secs | default parallelism=2


In [11]:
selcols = getIdFeats() + getCrmFeats() + getBillingFeats() + getMobSopoFeats() + getOrdersFeats()
now = datetime.now()


In [12]:
date_name = str('20190328')#str(now.year) + str(now.month).rjust(2, '0') + str(now.day).rjust(2, '0')
origin = '/user/hive/warehouse/tests_es.db/jvmm_amdocs_ids_'

In [13]:
# Fecha inicial va a ser un ciclo
# Fecha final es la fecha inicial+4 ciclos
trcycle_ini = '20181130'  #Training data
horizon = 4 #En ciclos
ttcycle_ini='20181231' #Test data
tr_ttdates = trcycle_ini + '_' + ttcycle_ini

In [14]:
# 1. TRAINING DATA
# 1.1. Loading training data

In [66]:
#inittrdf_ini = getFbbChurnUnlabeledCar(spark, origin,trcycle_ini ,selcols)

df = getFbbChurnLabeledCarCycles2(spark, origin, trcycle_ini, selcols, horizon = 4)

[Info getCarNumClienteDf] Tue Jul 16 12:10:43 2019 Size of the original df for 20181130: 13248445
[Info FbbChurn] Tue Jul 16 12:15:31 2019 Saving origindf_tmp to HDFS
[Info getCarNumClienteDf] Tue Jul 16 12:15:32 2019 Size of the origindf after load from HDFS: 13248445
[Info getCarNumClienteDf] Tue Jul 16 12:16:44 2019 Size of the numclidf for 20181130 - Num rows: 13248445 - Num columns: 101
[Info FbbChurn] Tue Jul 16 12:20:02 2019 Saving numclidf_tmp to HDFS
[Info get_billing_df] Tue Jul 16 12:20:03 2019 Starting the preparation of billing feats
[Info get_billing_df] Tue Jul 16 12:20:12 2019 Size of the df with billing feats: 5714359
[Info getCarNumClienteDf] Tue Jul 16 12:20:16 2019 Size of the tmpdf for 20181130 - Num rows: 13248445 - Num columns: 134
[Info get_mobile_spinners_df] Tue Jul 16 12:20:27 2019 Port-out data loaded for 20181130 with a total of 27468495 rows and 11448545 distinct NIFs
[Info get_mobile_spinners_df] Tue Jul 16 12:20:33 2019 Timing feats computed for a total 

In [16]:
inittrdf_ini.printSchema()

root
 |-- num_cliente: string (nullable = true)
 |-- nif_cliente: string (nullable = true)
 |-- rgu: string (nullable = true)
 |-- campo1: string (nullable = true)
 |-- msisdn: string (nullable = true)
 |-- msisdn_d: string (nullable = true)
 |-- mean_dias_desde_fx_srv_basic: double (nullable = false)
 |-- movil_services: long (nullable = true)
 |-- tv_services: long (nullable = true)
 |-- total_penal_cust_pending_n3_penal_amount: double (nullable = false)
 |-- x_formato_factura: string (nullable = true)
 |-- x_user_twitter: string (nullable = true)
 |-- total_penal_cust_pending_n4_penal_amount: double (nullable = false)
 |-- dias_desde_bam-movil_fx_first: double (nullable = false)
 |-- max_dias_desde_fx_trybuy_autom_tv: double (nullable = false)
 |-- total_price_tariff: double (nullable = false)
 |-- flg_robinson: string (nullable = true)
 |-- pvr_services: double (nullable = false)
 |-- num_tariff_redl: double (nullable = false)
 |-- dias_desde_fx_fbb_upgrade: double (nullable = fals

In [17]:
##Añadimos extra features

In [67]:
## Añadimos las extra features (lo hacemos aquí fuera porque ya están filtradas por rgu)
dfExtraFeat = spark.read.parquet('/data/udf/vf_es/churn/extra_feats_mod/extra_feats/year={}/month={}/day={}'
                                            .format(int(trcycle_ini[0:4]), int(trcycle_ini[4:6]), int(trcycle_ini[6:8])))


In [68]:
dfExtraFeatfbb = dfExtraFeat.join(df, ["num_cliente"], "leftsemi")


In [69]:
dfExtraFeatSel,selColumnas=addExtraFeats(dfExtraFeatfbb)

('[Info FbbChurn] Tue Jul 16 12:25:25 2019 The module for the Extra Feats has been run. Count for the table', 1978186)


In [70]:
dfillNa = fillNa(spark)
for kkey in dfillNa.keys():
    if kkey not in dfExtraFeatSel.columns:
        dfillNa.pop(kkey, None)

In [73]:
inittrdf = df.join(dfExtraFeatSel, ["msisdn","num_cliente",'rgu'], how="left").na.fill(dfillNa)

In [74]:
inittrdf.count()

1978186

In [26]:
len(dfExtraFeatSel.columns),len(inittrdf_ini.columns),len(inittrdf.columns)

(92, 238, 327)

In [27]:
horizon = 4
yearmonthday = '20181130'
cycle = 0
fini_tmp = yearmonthday
while cycle < horizon:
    yearmonthday_target = get_next_cycle(fini_tmp, str_fmt="%Y%m%d")
    cycle = cycle + 1
    fini_tmp = yearmonthday_target

yearmonth = yearmonthday[0:6]

In [28]:
fixporttr = getFixPortRequestsForCycleList(spark, yearmonthday, yearmonthday_target)
fixdxtr = getFbbDxsForCycleList(spark,yearmonthday, yearmonthday_target)

20181130
20181231
[Info getFixPortRequestsForMonth] Tue Jul 16 11:06:30 2019 Port-out requests for fixed services during period 20181130-20181231: 47422
[Info getFbbDxsForMonth] Tue Jul 16 11:06:43 2019 DXs for FBB services during the period: 20181130-20181231: 63345


In [64]:
def getFbbChurnLabeledCarCycles2(spark, origin, yearmonthday, selcols, horizon = 4):

    cycle = 0
    fini_tmp = yearmonthday
    while cycle < horizon:
        yearmonthday_target = get_next_cycle(fini_tmp, str_fmt="%Y%m%d")
        cycle = cycle + 1
        fini_tmp = yearmonthday_target

    yearmonth = yearmonthday[0:6]

    trfeatdf = getCarNumClienteDf(spark, origin, yearmonthday)

    print("[Info getFbbChurnLabeledCar] " + time.ctime() + " Samples for month " + yearmonthday + ": " + str(trfeatdf.count()))

    # Loading port-out requests and DXs
    # # labmonthlisttr = getMonthSeq(initportmonthtr, lastportmonthtr)

    # Las bajas de fibra pueden venir por:
    #- Solicitudes de baja de fijo
    fixporttr = getFixPortRequestsForCycleList(spark, yearmonthday, yearmonthday_target)
    #- Porque dejen de estar en la lista de clientes
    fixdxtr = getFbbDxsForCycleList(spark,yearmonthday, yearmonthday_target)

    # Labeling: FBB service is labeled as 1 if, during the next time window specified by the horizon, either the associated fixed service requested to be ported out or the FBB was disconnected
    window = Window.partitionBy("num_cliente")

    unbaltrdf = trfeatdf\
    .join(fixporttr, ['msisdn_d'], "left_outer")\
    .na.fill({'label_srv': 0.0})\
    .join(fixdxtr, ['msisdn'], "left_outer")\
    .na.fill({'label_dx': 0.0})\
    .withColumn('tmp', when((col('label_srv')==1.0) | (col('label_dx')==1.0), 1.0).otherwise(0.0))\
    .withColumn('label', sql_max('tmp').over(window))\
    .filter(col("rgu")=="fbb")\
    .select(selcols + ['label'])

    print("[Info getFbbChurnLabeledCar] " + time.ctime() + " Labeled samples for month " + yearmonth + ": " + str(unbaltrdf.count()))

    return unbaltrdf

In [87]:
def getFbbChurnLabeledCarCycles3(spark, origin, yearmonthday, selcols, horizon = 4):

    cycle = 0
    fini_tmp = yearmonthday
    while cycle < horizon:
        yearmonthday_target = get_next_cycle(fini_tmp, str_fmt="%Y%m%d")
        cycle = cycle + 1
        fini_tmp = yearmonthday_target

    yearmonth = yearmonthday[0:6]

    trfeatdf = getCarNumClienteDf(spark, origin, yearmonthday)

    print("[Info getFbbChurnLabeledCar] " + time.ctime() + " Samples for month " + yearmonthday + ": " + str(trfeatdf.count()))

    # Loading port-out requests and DXs
    # # labmonthlisttr = getMonthSeq(initportmonthtr, lastportmonthtr)

    # Las bajas de fibra pueden venir por:
    #- Solicitudes de baja de fijo
    fixporttr = getFixPortRequestsForCycleList(spark, yearmonthday, yearmonthday_target)
    #- Porque dejen de estar en la lista de clientes
    fixdxtr = getFbbDxsForCycleList(spark,yearmonthday, yearmonthday_target)

    # Labeling: FBB service is labeled as 1 if, during the next time window specified by the horizon, either the associated fixed service requested to be ported out or the FBB was disconnected
    window = Window.partitionBy("num_cliente")

    unbaltrdf = trfeatdf\
    .join(fixporttr, ['msisdn_d'], "left_outer")\
    .na.fill({'label_srv': 0.0})\
    .join(fixdxtr, ['msisdn'], "left_outer")\
    .na.fill({'label_dx': 0.0})\
    .withColumn('tmp', when((col('label_dx')==1.0), 1.0).otherwise(0.0))\
    .withColumn('label', sql_max('tmp').over(window))\
    .filter(col("rgu")=="fbb")\
    .select(selcols + ['label'])

    print("[Info getFbbChurnLabeledCar] " + time.ctime() + " Labeled samples for month " + yearmonth + ": " + str(unbaltrdf.count()))

    return unbaltrdf

In [53]:
inittrdf.count()

13248445

In [76]:
inittrdf.groupBy('label').agg(count('*')).show()

+-----+--------+
|label|count(1)|
+-----+--------+
|  0.0| 1956543|
|  1.0|   21643|
+-----+--------+



In [79]:
[unbaltrdf, valdf] = inittrdf.randomSplit([0.7, 0.3], 1234)

In [31]:
len(inittrdf.columns), len(trfeatdf.columns)

(327, 238)

In [50]:
unbaltrdf.count()

1385997

In [ ]:
# 1.2. Balanced df for training

In [80]:
trdf = balance_df2(unbaltrdf, 'label')

[Info balance_df2] Dataframe has been balanced - Total number of rows is 30459


In [81]:
trdf.groupBy('label').agg(count('*')).show()

+-----+--------+
|label|count(1)|
+-----+--------+
|  0.0|   15306|
|  1.0|   15153|
+-----+--------+



In [82]:
allFeats = trdf.columns

# Getting only the numeric variables
catCols = [item[0] for item in trdf.dtypes if item[1].startswith('string')]
numerical_feats = list(set(allFeats) - set(list(
        set().union(getIdFeats(), getIdFeats_tr(), getNoInputFeats(), catCols, [c + "_enc" for c in getCatFeatsCrm()],
                    ["label"]))))

noninf_feats = getNonInfFeats(trdf, numerical_feats)

[Info FbbChurn] 1/2 Selecting the features that add information to the model
[Info FbbChurn] 2/2 Selecting the features that add information to the model


In [83]:
featCols = list(set(numerical_feats) - set(noninf_feats))

assembler = VectorAssembler(inputCols=featCols, outputCol="features")

classifier = RandomForestClassifier(featuresCol="features", \
                                        labelCol="label", \
                                        maxDepth=20, \
                                        maxBins=32, \
                                        minInstancesPerNode=100, \
                                        impurity="entropy", \
                                        featureSubsetStrategy="sqrt", \
                                        subsamplingRate=0.85, minInfoGain = 0.001, \
                                        numTrees=800, \
                                        seed=1234)  

pipeline = Pipeline(stages=[assembler, classifier])
model = pipeline.fit(trdf)

In [84]:
calibmodel = get_calibration_function2(spark, model, valdf, 'label', 10)

getScore = udf(lambda prob: float(prob[1]), DoubleType())

# Train
tr_preds_df = model.transform(trdf).withColumn("model_score", getScore(col("probability")).cast(DoubleType()))
tr_calib_preds_df = calibmodel[0].transform(tr_preds_df)

[Info Calibration] Training - Num samples in the target class 6490.0
[Info Calibration] Training 0.140616770625, 0.00169491525424
[Info Calibration] Training 0.223732049807, 0.00832049306626
[Info Calibration] Training 0.306847328989, 0.0281972265023
[Info Calibration] Training 0.389962608171, 0.0790446841294
[Info Calibration] Training 0.473077887353, 0.217257318952
[Info Calibration] Training 0.556193166535, 0.420647149461
[Info Calibration] Training 0.639308445718, 0.645762711864
[Info Calibration] Training 0.7224237249, 0.862095531587
[Info Calibration] Training 0.805539004082, 0.985824345146
+-------------------+--------------------+
|        model_score|         target_prob|
+-------------------+--------------------+
| 0.1406167706248943|0.001694915254237288|
| 0.2237320498070065|0.008320493066255779|
|0.30684732898911865|0.028197226502311247|
|0.38996260817123085| 0.07904468412942989|
|0.47307788735334305|  0.2172573189522342|
| 0.5561931665354553|  0.4206471494607088|
| 0.63930

In [85]:
trPredictionAndLabels = tr_calib_preds_df.select(['calib_model_score', 'label']).rdd.map(lambda r: (r['calib_model_score'], r['label']))
trmetrics = BinaryClassificationMetrics(trPredictionAndLabels)

In [86]:
trmetrics.areaUnderROC

0.8081188067089619

In [ ]:
df2 = getFbbChurnLabeledCarCycles3(spark, origin, trcycle_ini, selcols, horizon = 4)

[Info getCarNumClienteDf] Tue Jul 16 15:01:39 2019 Size of the original df for 20181130: 13248445
[Info FbbChurn] Tue Jul 16 15:05:26 2019 Saving origindf_tmp to HDFS
[Info getCarNumClienteDf] Tue Jul 16 15:05:27 2019 Size of the origindf after load from HDFS: 13248445
[Info getCarNumClienteDf] Tue Jul 16 15:06:39 2019 Size of the numclidf for 20181130 - Num rows: 13248445 - Num columns: 101
[Info FbbChurn] Tue Jul 16 15:09:26 2019 Saving numclidf_tmp to HDFS
[Info get_billing_df] Tue Jul 16 15:09:26 2019 Starting the preparation of billing feats


In [95]:
dfExtraFeatfbb = dfExtraFeat.join(df2, ["num_cliente"], "leftsemi")

In [96]:
dfExtraFeatSel,selColumnas=addExtraFeats(dfExtraFeatfbb)

('[Info FbbChurn] Wed Jul 17 07:49:23 2019 The module for the Extra Feats has been run. Count for the table', 1978186)


In [97]:
dfillNa = fillNa(spark)
for kkey in dfillNa.keys():
    if kkey not in dfExtraFeatSel.columns:
        dfillNa.pop(kkey, None)

In [98]:
inittrdf = df2.join(dfExtraFeatSel, ["msisdn","num_cliente",'rgu'], how="left").na.fill(dfillNa)

In [99]:
inittrdf.count()

1978186

In [100]:
inittrdf.groupBy('label').agg(count('*')).show()

+-----+--------+
|label|count(1)|
+-----+--------+
|  0.0| 1914837|
|  1.0|   63349|
+-----+--------+



In [101]:
[unbaltrdf, valdf] = inittrdf.randomSplit([0.7, 0.3], 1234)

In [102]:
unbaltrdf.count()

1385871

In [103]:
trdf = balance_df2(unbaltrdf, 'label')

[Info balance_df2] Dataframe has been balanced - Total number of rows is 88454


In [104]:
trdf.groupBy('label').agg(count('*')).show()

+-----+--------+
|label|count(1)|
+-----+--------+
|  0.0|   44175|
|  1.0|   44279|
+-----+--------+



In [105]:
allFeats = trdf.columns

# Getting only the numeric variables
catCols = [item[0] for item in trdf.dtypes if item[1].startswith('string')]
numerical_feats = list(set(allFeats) - set(list(
        set().union(getIdFeats(), getIdFeats_tr(), getNoInputFeats(), catCols, [c + "_enc" for c in getCatFeatsCrm()],
                    ["label"]))))

noninf_feats = getNonInfFeats(trdf, numerical_feats)

[Info FbbChurn] 1/2 Selecting the features that add information to the model
[Info FbbChurn] 2/2 Selecting the features that add information to the model


In [106]:
featCols = list(set(numerical_feats) - set(noninf_feats))

assembler_dx = VectorAssembler(inputCols=featCols, outputCol="features")

classifier_dx = RandomForestClassifier(featuresCol="features", \
                                        labelCol="label", \
                                        maxDepth=20, \
                                        maxBins=32, \
                                        minInstancesPerNode=100, \
                                        impurity="entropy", \
                                        featureSubsetStrategy="sqrt", \
                                        subsamplingRate=0.85, minInfoGain = 0.001, \
                                        numTrees=800, \
                                        seed=1234)  

pipeline_dx = Pipeline(stages=[assembler_dx, classifier_dx])
model_dx = pipeline_dx.fit(trdf)

In [107]:
calibmodel_dx = get_calibration_function2(spark, model_dx, valdf, 'label', 10)

getScore = udf(lambda prob: float(prob[1]), DoubleType())

# Train
tr_preds_df_dx = model_dx.transform(trdf).withColumn("model_score", getScore(col("probability")).cast(DoubleType()))
tr_calib_preds_df_dx = calibmodel_dx[0].transform(tr_preds_df_dx)

[Info Calibration] Training - Num samples in the target class 19070.0
[Info Calibration] Training 0.154809694588, 0.00209753539591
[Info Calibration] Training 0.245802397268, 0.0149973780808
[Info Calibration] Training 0.336795099947, 0.085631882538
[Info Calibration] Training 0.427787802627, 0.218668065024
[Info Calibration] Training 0.518780505306, 0.3843733613
[Info Calibration] Training 0.609773207986, 0.577608809649
[Info Calibration] Training 0.700765910665, 0.737021499738
[Info Calibration] Training 0.791758613345, 0.8642370215
[Info Calibration] Training 0.882751316024, 0.940010487677
+-------------------+--------------------+
|        model_score|         target_prob|
+-------------------+--------------------+
|0.15480969458839328|0.002097535395909806|
|0.24580239726784703|0.014997378080755113|
| 0.3367950999473008| 0.08563188253801783|
|0.42778780262675453| 0.21866806502359729|
| 0.5187805053062082| 0.38437336130047195|
|  0.609773207985662|  0.5776088096486628|
| 0.700765910

In [108]:
trPredictionAndLabels_dx = tr_calib_preds_df_dx.select(['calib_model_score', 'label']).rdd.map(lambda r: (r['calib_model_score'], r['label']))
trmetrics_dx = BinaryClassificationMetrics(trPredictionAndLabels_dx)

In [109]:
trmetrics_dx.areaUnderROC

0.7830164412662808

# Test

In [110]:
df_test_dx = getFbbChurnLabeledCarCycles3(spark, origin, ttcycle_ini, selcols, horizon = 4)

[Info getCarNumClienteDf] Wed Jul 17 11:15:36 2019 Size of the original df for 20181231: 13683905
[Info FbbChurn] Wed Jul 17 11:20:08 2019 Saving origindf_tmp to HDFS
[Info getCarNumClienteDf] Wed Jul 17 11:20:08 2019 Size of the origindf after load from HDFS: 13683905
[Info getCarNumClienteDf] Wed Jul 17 11:21:28 2019 Size of the numclidf for 20181231 - Num rows: 13683905 - Num columns: 101
[Info FbbChurn] Wed Jul 17 11:24:59 2019 Saving numclidf_tmp to HDFS
[Info get_billing_df] Wed Jul 17 11:24:59 2019 Starting the preparation of billing feats
[Info get_billing_df] Wed Jul 17 11:25:35 2019 Size of the df with billing feats: 5825804
[Info getCarNumClienteDf] Wed Jul 17 11:25:42 2019 Size of the tmpdf for 20181231 - Num rows: 13683905 - Num columns: 134
[Info get_mobile_spinners_df] Wed Jul 17 11:25:54 2019 Port-out data loaded for 20181231 with a total of 27844643 rows and 11499038 distinct NIFs
[Info get_mobile_spinners_df] Wed Jul 17 11:26:17 2019 Timing feats computed for a total 

In [111]:
df_test_port = getFbbChurnLabeledCarCycles2(spark, origin, ttcycle_ini, selcols, horizon = 4)

[Info getCarNumClienteDf] Wed Jul 17 11:35:30 2019 Size of the original df for 20181231: 13683905
[Info FbbChurn] Wed Jul 17 11:41:47 2019 Saving origindf_tmp to HDFS
[Info getCarNumClienteDf] Wed Jul 17 11:41:47 2019 Size of the origindf after load from HDFS: 13683905
[Info getCarNumClienteDf] Wed Jul 17 11:43:02 2019 Size of the numclidf for 20181231 - Num rows: 13683905 - Num columns: 101
[Info FbbChurn] Wed Jul 17 11:46:44 2019 Saving numclidf_tmp to HDFS
[Info get_billing_df] Wed Jul 17 11:46:44 2019 Starting the preparation of billing feats
[Info get_billing_df] Wed Jul 17 11:46:57 2019 Size of the df with billing feats: 5825804
[Info getCarNumClienteDf] Wed Jul 17 11:47:07 2019 Size of the tmpdf for 20181231 - Num rows: 13683905 - Num columns: 134
[Info get_mobile_spinners_df] Wed Jul 17 11:47:23 2019 Port-out data loaded for 20181231 with a total of 27844643 rows and 11499038 distinct NIFs
[Info get_mobile_spinners_df] Wed Jul 17 11:47:31 2019 Timing feats computed for a total 

In [112]:
dfExtraFeat_tt = spark.read.parquet('/data/udf/vf_es/churn/extra_feats_mod/extra_feats/year={}/month={}/day={}'
                                            .format(int(ttcycle_ini[0:4]), int(ttcycle_ini[4:6]), int(ttcycle_ini[6:8])))


In [113]:
dfExtraFeatfbb_tt_dx = dfExtraFeat_tt.join(df_test_dx, ["num_cliente"], "leftsemi")

In [114]:
dfExtraFeatfbb_tt_port = dfExtraFeat_tt.join(df_test_port, ["num_cliente"], "leftsemi")

In [115]:
dfExtraFeatSel_tt_dx,selColumnas=addExtraFeats(dfExtraFeatfbb_tt_dx)

Py4JJavaError: An error occurred while calling o20097.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 8674.0 failed 4 times, most recent failure: Lost task 1.3 in stage 8674.0 (TID 984770, vgddp647hr.dc.sedc.internal.vodafone.com, executor 209): java.io.FileNotFoundException: File does not exist: hdfs://nameservice1/data/udf/vf_es/churn/fbb_tmp/numclidf_tmp_20181231/part-00154-00b1bfeb-7325-4f98-ac8a-dcc79a92cf52-c000.snappy.parquet
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:131)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:182)
	at org.apache.spark.sql.execution.datasour

In [ ]:
dfExtraFeatSel_tt_port,selColumnas=addExtraFeats(dfExtraFeatfbb_tt_port)

In [ ]:
initttdf_dx = df_test_dx.join(dfExtraFeatSel_tt_dx, ["msisdn","num_cliente",'rgu'], how="left").na.fill(dfillNa)

In [ ]:
initttdf_port = df_test_port.join(dfExtraFeatSel_tt_port, ["msisdn","num_cliente",'rgu'], how="left").na.fill(dfillNa)

In [101]:
ttdf_ini = getFbbChurnLabeledCarCycles(spark, origin,ttcycle_ini,selcols, horizon)

[Info getCarNumClienteDf] Wed May 29 14:55:23 2019 Size of the original df for 20181231: 13683905
[Info FbbChurn] Wed May 29 15:03:03 2019 Saving origindf_tmp to HDFS
[Info getCarNumClienteDf] Wed May 29 15:03:04 2019 Size of the origindf after load from HDFS: 13683905
[Info getCarNumClienteDf] Wed May 29 15:04:31 2019 Size of the numclidf for 20181231 - Num rows: 2050838 - Num columns: 101
[Info FbbChurn] Wed May 29 15:07:33 2019 Saving numclidf_tmp to HDFS
[Info get_billing_df] Wed May 29 15:07:34 2019 Starting the preparation of billing feats
[Info get_billing_df] Wed May 29 15:08:08 2019 Size of the df with billing feats: 5825804
[Info getCarNumClienteDf] Wed May 29 15:08:29 2019 Size of the tmpdf for 20181231 - Num rows: 2050838 - Num columns: 134
[Info get_mobile_spinners_df] Wed May 29 15:08:57 2019 Port-out data loaded for 20181231 with a total of 27844643 rows and 11499038 distinct NIFs
[Info get_mobile_spinners_df] Wed May 29 15:09:34 2019 Timing feats computed for a total of

In [ ]:
##Añadimos extra features

In [102]:
dfExtraFeat_tt = spark.read.parquet('/data/udf/vf_es/churn/extra_feats_mod/extra_feats/year={}/month={}/day={}'
                                            .format(int(ttcycle_ini[0:4]), int(ttcycle_ini[4:6]), int(ttcycle_ini[6:8])))


In [103]:
dfExtraFeatfbb_tt=dfExtraFeat_tt.join(ttdf_ini.select('num_cliente'),on='num_cliente',how='right')
print(dfExtraFeatfbb_tt.select('num_cliente').distinct().count(),ttdf_ini.select('num_cliente').distinct().count()) 


(2050764, 2050764)


In [104]:
dfExtraFeat_ttSel, selColumnas = addExtraFeats(dfExtraFeatfbb_tt)

('[Info FbbChurn] Wed May 29 15:22:21 2019 The module for the Extra Feats has been run. Count for the table', 2050986)


In [105]:
dfillNa = fillNa(spark)
for kkey in dfillNa.keys():
    if kkey not in dfExtraFeat_ttSel.columns:
        dfillNa.pop(kkey, None)

In [106]:
ttdf = ttdf_ini.join(dfExtraFeat_ttSel, ["msisdn", "num_cliente", 'rgu'], how="left").na.fill(dfillNa)

In [ ]:
# ***************************************************************************************
# ***************************************************************************************
#path = "/data/udf/vf_es/churn/fbb_tmp/ttdf_" + tr_ttdates
#ttdf = spark.read.parquet(path)
# ***************************************************************************************
# ***************************************************************************************

In [ ]:
# 3. MODELLING

In [209]:
vector_feats = []
for i in range(0,num_tr_dfs):
    featCols = list(set(numerical_feats) - set(noninf_feats_vec[i]))
    vector_feats.append(featCols)

In [210]:
assembler_vec = []
for i in range(0,num_tr_dfs):
    assembler = VectorAssembler(inputCols = vector_feats[i], outputCol = "features")
    assembler_vec.append(assembler)

In [211]:
classifier = RandomForestClassifier(featuresCol="features",\
        labelCol="label",\
        maxDepth=12,\
        maxBins=32,\
        minInstancesPerNode=200,\
        impurity="gini",\
        featureSubsetStrategy="sqrt",\
        subsamplingRate=0.7,\
        numTrees=800,\
        seed = 1234,)

In [213]:
pipeline_vec = []
for i in range(0,num_tr_dfs):
    pipeline = Pipeline(stages= [assembler_vec[i], classifier])
    pipeline_vec.append(pipeline)

In [214]:
models_vec = []
for i in range(0,num_tr_dfs):
    model = pipeline_vec[i].fit(vector_train[i])
    models_vec.append(model)
    print(i)

0
1
2
3
4


In [215]:
def get_calibration_function_vec(spark, model, valdf, labelcol, numpoints = 10, num = 0):
    import numpy as np
    score_name = "model_score_" + str(num)
    score_calib_name ='calib_model_score_' + str(num)
    getScore = udf(lambda prob: float(prob[1]), DoubleType())
    valpredsdf = model.transform(valdf).withColumn(score_name, getScore(col("probability")).cast(DoubleType())).select([score_name, labelcol])
    minprob = valpredsdf.select(sql_min(score_name).alias('min')).rdd.collect()[0]['min']
    maxprob = valpredsdf.select(sql_max(score_name).alias('max')).rdd.collect()[0]['max']
    numlabels = float(valpredsdf.filter(col(labelcol) == 1.0).count())
    print "[Info Calibration] Training - Num samples in the target class " + str(numlabels)
    delta = float(maxprob - minprob)/float(numpoints)
    ths = list([(minprob + i*delta) for i in list(np.arange(1,numpoints,step=1))])
    samplepoints = [(float(i), float(valpredsdf.filter((col(score_name) <= i) & (col(labelcol) == 1.0)).count())/numlabels) for i in ths]
    # samplepoints = [(float(i), valpredsdf.filter(col('model_score') <= i &).select(sql_avg(labelcol).alias('rate')).rdd.collect()[0]['rate']) for i in ths]
    for pair in samplepoints:
        print "[Info Calibration] Training " + str(pair[0]) + ", " + str(pair[1])
    sampledf = spark.createDataFrame(samplepoints, [score_name, 'target_prob'])
    sampledf.show()
    print "[Info Calibration] Samples to implement the calibration model showed above"
    ir = IsotonicRegression(featuresCol=score_name, labelCol='target_prob', predictionCol=score_calib_name)
    irmodel = ir.fit(sampledf)
    return (irmodel, samplepoints)

In [ ]:
# ***************************************************************************************
# ***************************************************************************************
# path2 = '/data/udf/vf_es/churn/fbb_tmp/valdf_' + tr_ttdates
# valdf = spark.read.parquet(path2)
# ***************************************************************************************
# ***************************************************************************************

In [217]:
# EVALUATION
# Calibration
calibration_vec = []
for i in range(0,num_tr_dfs):
    calibmodel = get_calibration_function_vec(spark, models_vec[i], valdf, 'label', 10, i)
    calibration_vec.append(calibmodel)
    print(i)

[Info Calibration] Training - Num samples in the target class 19070.0
[Info Calibration] Training 0.174235204208, 0.00272679601468
[Info Calibration] Training 0.260449978173, 0.0162034609334
[Info Calibration] Training 0.346664752138, 0.0958049292082
[Info Calibration] Training 0.432879526103, 0.224908232826
[Info Calibration] Training 0.519094300069, 0.392711064499
[Info Calibration] Training 0.605309074034, 0.578605138962
[Info Calibration] Training 0.691523847999, 0.737703198741
[Info Calibration] Training 0.777738621964, 0.868274777137
[Info Calibration] Training 0.86395339593, 0.950340849502
+-------------------+--------------------+
|      model_score_0|         target_prob|
+-------------------+--------------------+
|0.17423520420768662|0.002726796014682...|
| 0.2604499781729195| 0.01620346093340325|
| 0.3466647521381523| 0.09580492920818039|
| 0.4328795261033851| 0.22490823282642894|
| 0.5190943000686179| 0.39271106449921345|
| 0.6053090740338508|    0.57860513896172|
| 0.69152

In [218]:
getScore = udf(lambda prob: float(prob[1]), DoubleType())

In [220]:
# Train
tr_calib_vec = []
for i in range(0, num_tr_dfs):
    tr_preds_df = models_vec[i].transform(vector_train[i]).withColumn("model_score_" + str(i), getScore(col("probability")).cast(DoubleType()))
    tr_calib_preds_df = calibration_vec[i][0].transform(tr_preds_df)
    tr_calib_vec.append(tr_calib_preds_df)
    print(i)

0
1
2
3
4


In [221]:
tr_metrics_vec = []
# Train evaluation
#tr_calib_vec = []
for i in range(0, num_tr_dfs):#'calib_model_score_' + str(num)
    trPredictionAndLabels = tr_calib_vec[i].select(['calib_model_score_' + str(i), 'label']).rdd.map(lambda r: (r['calib_model_score_' + str(i)], r['label']))
    trmetrics = BinaryClassificationMetrics(trPredictionAndLabels)
    print(trmetrics.areaUnderROC)
    tr_metrics_vec.append(trmetrics)
    print(i)

0.754406933206
0
0.755106476011
1
0.753097394074
2
0.755560008586
3
0.755012309843
4


In [222]:
# Test eval
df_aux = ttdf
for i in range(0, num_tr_dfs):
    df_aux = models_vec[i].transform(df_aux).withColumn("model_score_" + str(i), getScore(col("probability")).cast(DoubleType())).drop(col('probability'))
    # tt_preds_df
    df_aux = calibration_vec[i][0].transform(df_aux.drop(col('features')).drop(col('prediction')).drop(col('rawPrediction')))
    print(i)
    
save_dir = 'tests_es.asaezco_fbb_improved_' + str(20190131)
df_aux.write.format('parquet').mode('overwrite').saveAsTable(save_dir)

0
1
2
3
4


In [223]:
df_aux.printSchema()

root
 |-- msisdn: string (nullable = true)
 |-- num_cliente: string (nullable = true)
 |-- rgu: string (nullable = true)
 |-- campo1: string (nullable = true)
 |-- msisdn_d: string (nullable = true)
 |-- nif_cliente: string (nullable = true)
 |-- mean_dias_desde_fx_srv_basic: double (nullable = false)
 |-- movil_services: long (nullable = true)
 |-- tv_services: long (nullable = true)
 |-- total_penal_cust_pending_n3_penal_amount: double (nullable = false)
 |-- x_formato_factura: string (nullable = true)
 |-- x_user_twitter: string (nullable = true)
 |-- total_penal_cust_pending_n4_penal_amount: double (nullable = false)
 |-- dias_desde_bam-movil_fx_first: double (nullable = false)
 |-- max_dias_desde_fx_trybuy_autom_tv: double (nullable = false)
 |-- total_price_tariff: double (nullable = false)
 |-- flg_robinson: string (nullable = true)
 |-- pvr_services: double (nullable = false)
 |-- num_tariff_redl: double (nullable = false)
 |-- dias_desde_fx_fbb_upgrade: double (nullable = fals

In [65]:
# Evaluation
ttPredictionAndLabels = tt_calib_preds_df.select(['calib_model_score', 'label']).rdd.map(lambda r: (r['calib_model_score'], r['label']))
ttmetrics = BinaryClassificationMetrics(ttPredictionAndLabels)

In [66]:
print("[Info FbbChurn] " + time.ctime() + " Area under ROC(tr) = " + str(trmetrics.areaUnderROC) + " - Area under ROC(tt) = " + str(ttmetrics.areaUnderROC))


[Info FbbChurn] Wed May 29 11:42:15 2019 Area under ROC(tr) = 0.760427050776 - Area under ROC(tt) = 0.714090517079


In [224]:
for i in range(0, num_tr_dfs):
    ttPredictionAndLabels = df_aux.select(['calib_model_score_' + str(i), 'label']).rdd.map(lambda r: (r['calib_model_score_' + str(i)], r['label']))
    ttmetrics = BinaryClassificationMetrics(ttPredictionAndLabels)
    #print(ttmetrics.areaUnderROC)
    print(i)
    print "Model {}: Area under ROC(tr) = {}".format(i, str(tr_metrics_vec[i].areaUnderROC))
    print "Model {}: Area under ROC(tt) = {}".format(i, str(ttmetrics.areaUnderROC))

0
Model 0: Area under ROC(tr) = 0.754406933206
Model 0: Area under ROC(tt) = 0.713514753724
1
Model 1: Area under ROC(tr) = 0.755106476011
Model 1: Area under ROC(tt) = 0.713128028314
2
Model 2: Area under ROC(tr) = 0.753097394074
Model 2: Area under ROC(tt) = 0.712410491476
3
Model 3: Area under ROC(tr) = 0.755560008586
Model 3: Area under ROC(tt) = 0.712465860634
4
Model 4: Area under ROC(tr) = 0.755012309843
Model 4: Area under ROC(tt) = 0.713019849073


In [234]:
cols_scores

['calib_model_score_0',
 'calib_model_score_1',
 'calib_model_score_2',
 'calib_model_score_3',
 'calib_model_score_4']

In [235]:
cols_scores = [name_ for name_ in df_aux.columns if name_.startswith('calib_model_')]

from pyspark.sql.functions import greatest, least, mean, sum

df_aux2 = df_aux.withColumn('maximo', greatest(*cols_scores)).withColumn('minimo', least(*cols_scores))\
.withColumn('medio', (df_aux['calib_model_score_0'] + df_aux['calib_model_score_1'] + df_aux['calib_model_score_2'] + df_aux['calib_model_score_3'] + df_aux['calib_model_score_4'])/5)#.withColumn('minimo', least(*cols_scores))

In [229]:
for name in ['maximo','minimo']:
    tt_ensemble = df_aux2.select([name, 'label']).rdd.map(lambda r: (r[name], r['label']))
    ttmetrics = BinaryClassificationMetrics(tt_ensemble)
    print "Model {}: Area under ROC(tt) = {}".format(i, str(ttmetrics.areaUnderROC))

Model 4: Area under ROC(tt) = 0.712157131246
Model 4: Area under ROC(tt) = 0.713841078181


In [156]:
for i in range(0,2):
    print(tr_metrics_vec[i].areaUnderROC)

0.767949194081
1.0


In [236]:
for name in ['medio']:
    tt_ensemble = df_aux2.select([name, 'label']).rdd.map(lambda r: (r[name], r['label']))
    ttmetrics = BinaryClassificationMetrics(tt_ensemble)
    print "Model {}: Area under ROC(tt) = {}".format(i, str(ttmetrics.areaUnderROC))

Model 4: Area under ROC(tt) = 0.713220048003


In [249]:
tr = tr_1s.union(vec_selected_0s[num_tr_dfs].limit(num_1s))

In [250]:
tr.count()

Py4JJavaError: An error occurred while calling o32030.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 37 in stage 29996.0 failed 4 times, most recent failure: Lost task 37.3 in stage 29996.0 (TID 4397224, vgddp601hr.dc.sedc.internal.vodafone.com, executor 364): java.io.FileNotFoundException: File does not exist: hdfs://nameservice1/data/udf/vf_es/churn/fbb_tmp/numclidf_tmp_20181130/part-00115-96c44731-02fb-4b41-bb45-ac2d0396c457-c000.snappy.parquet
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:131)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:182)
	at org.apache.spark.sql.execution.dat

In [238]:
df_aux_tr = tr
for i in range(0, num_tr_dfs):
    df_aux_tr = models_vec[i].transform(df_aux_tr).withColumn("model_score_" + str(i), getScore(col("probability")).cast(DoubleType())).drop(col('probability'))
    # tt_preds_df
    df_aux_tr = calibration_vec[i][0].transform(df_aux_tr.drop(col('features')).drop(col('prediction')).drop(col('rawPrediction')))
    print(i)

0
1
2
3
4


In [239]:
features_ = cols_scores
assembler = VectorAssembler(inputCols = features_, outputCol = "features")

classifier = RandomForestClassifier(featuresCol="features",\
        labelCol="label",\
        maxDepth=10,\
        maxBins=32,\
        minInstancesPerNode=200,\
        impurity="gini",\
        featureSubsetStrategy="sqrt",\
        subsamplingRate=0.7,\
        numTrees=800,\
        seed = 1234,)

pipeline = Pipeline(stages= [assembler, classifier])

In [243]:
tr.count()

Py4JJavaError: An error occurred while calling o31302.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 8 in stage 29896.0 failed 4 times, most recent failure: Lost task 8.3 in stage 29896.0 (TID 4397001, vgddp426hr.dc.sedc.internal.vodafone.com, executor 361): java.io.FileNotFoundException: File does not exist: hdfs://nameservice1/data/udf/vf_es/churn/fbb_tmp/numclidf_tmp_20181130/part-00175-96c44731-02fb-4b41-bb45-ac2d0396c457-c000.snappy.parquet
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:131)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:182)
	at org.apache.spark.sql.execution.datas

In [240]:
model_final = pipeline.fit(df_aux_tr)

Py4JJavaError: An error occurred while calling o31774.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 8 in stage 29790.0 failed 4 times, most recent failure: Lost task 8.3 in stage 29790.0 (TID 4396812, vgddp426hr.dc.sedc.internal.vodafone.com, executor 361): java.io.FileNotFoundException: File does not exist: hdfs://nameservice1/data/udf/vf_es/churn/fbb_tmp/numclidf_tmp_20181130/part-00175-96c44731-02fb-4b41-bb45-ac2d0396c457-c000.snappy.parquet
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:131)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:182)
	at org.apache.spark.sql.execution.datasou

In [ ]:
.transform(tr).withColumn("model_score", getScore(col("probability")).cast(DoubleType()))